## Abstraction layer to work with any LLM API

This Python code defines a class called `MyLLM` that provides an interface to interact with three different language model APIs: OpenAI, Cohere, and Anthropic. Here's a breakdown of the code:

### Import Statements
```python
import openai
import cohere
import anthropic
```
These import the necessary libraries for each respective API.

### Class Definition
- `__init__` initializes the class with the type of API (`api_type`), an API key (`api_key`), and the model to use.
- Depending on the `api_type` provided, it creates a client for the corresponding API and assigns it to `self.client`.

### Generate Response Method
- `generate_response` generates a response based on the messages provided.
- It constructs the appropriate prompt for the selected API and then makes an API call to generate a response.
- The response format and method to access the generated text differ for each API.

### Helper Methods for Constructing Prompts
- These methods construct the prompt for the respective APIs from the provided messages.
- For Cohere and Anthropic, it concatenates messages into a single string with appropriate roles (`System`, `User`, `Assistant`).
- The prompt ends with `Assistant:` to indicate that the next text generated should be from the assistant.

### Summary
This class provides a unified interface to generate responses using different language model APIs. The key steps include:
1. Initializing the class with the appropriate API client.
2. Constructing prompts suitable for the selected API.
3. Making API calls to generate responses and extracting the relevant content from the responses.

In [3]:
import openai
import cohere
import anthropic

class MyLLM:
    def __init__(self, api_type, api_key, model):
        self.api_type = api_type
        self.api_key = api_key
        self.model = model
        
        if api_type == 'openai':
            self.client = openai.OpenAI()
            self.client.api_key = api_key
        elif api_type == 'cohere':
            self.client = cohere.Client(api_key=api_key)
        elif api_type == 'anthropic':
            self.client = anthropic.Client(api_key=api_key)
        else:
            raise ValueError("Unsupported API type. Choose 'openai', 'cohere', or 'anthropic'.")

    def generate_response(self, messages, temperature=0.0, max_tokens=1000):
        if self.api_type == 'openai':
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].message.content
        elif self.api_type == 'cohere':
            response = self.client.chat(
                model=self.model,
                message=messages,
                max_tokens=max_tokens,
                temperature=temperature,
                stop_sequences=["USER:", "SYSTEM:", "CHATBOT:"]
            )
            return response.text
        elif self.api_type == 'anthropic':
            response = self.client.completions.create(
                model=self.model,
                prompt=messages,
                max_tokens_to_sample=max_tokens,
                temperature=temperature,
                stop_sequences=["User:", "System:", "Assistant:"]
            )
            return response["completion"].strip()


## Main models available for each API

## OpenAI Models

| Model Name          | Code                       | Description                            |
|---------------------|----------------------------|----------------------------------------|
| GPT-4o              | gpt-4o                     | Fastest and most affordable.           |
| GPT-4 Turbo         | gpt-4-turbo                | Similar to GPT-4o but slower.          |
| GPT-3.5 Turbo       | gpt-3.5-turbo              | Fast and inexpensive.                  |
| DALL·E 3            | dall-e-3                   | Generates and edits images.            |
| DALL·E 2            | dall-e-2                   | Generates and edits images.            |
| TTS                 | tts-1                      | Text to speech.                        |
| TTS HD              | tts-1-hd                   | High-definition text to speech.        |
| Whisper             | whisper-1                  | Audio to text.                         |
| Embeddings Large    | text-embedding-3-large     | Text to embeddings.                    |
| Embeddings Small    | text-embedding-3-small     | Text to embeddings.                    |
| Embeddings Ada      | text-embedding-ada-002     | Text to embeddings.                    |
| Moderation Latest   | text-moderation-latest     | Detects sensitive text.                |
| Moderation Stable   | text-moderation-stable     | Detects sensitive text.                |
| Moderation 007      | text-moderation-007        | Detects sensitive text.                |
| GPT Base Babbage    | babbage-002                | Base language model.                   |
| GPT Base Davinci    | davinci-002                | Base language model.                   |

## Cohere API Models

| Model Name                  | Code                        | Description                            |
|-----------------------------|-----------------------------|----------------------------------------|
| Command R+                  | command-r-plus              | High-quality instruction model.        |
| Command R                   | command-r                   | High-quality instruction model.        |
| Command                     | command                     | High-quality instruction model.        |
| Command Light               | command-light               | Fast, smaller instruction model.       |
| Embed English v3.0          | embed-english-v3.0          | English embeddings.                    |
| Embed English Light v3.0    | embed-english-light-v3.0    | English embeddings.                    |
| Embed Multilingual v3.0     | embed-multilingual-v3.0     | Multilingual embeddings.               |
| Embed Multilingual Light v3.0 | embed-multilingual-light-v3.0 | Multilingual embeddings.               |
| Embed English v2.0          | embed-english-v2.0          | English embeddings.                    |
| Embed English Light v2.0    | embed-english-light-v2.0    | English embeddings.                    |
| Embed Multilingual v2.0     | embed-multilingual-v2.0     | Multilingual embeddings.               |

## Anthropic API Models

| Model Name            | Code                       | Description                            |
|-----------------------|----------------------------|----------------------------------------|
| Claude 3.5 Sonnet     | claude-3-5-sonnet-20240620 | Latest Claude 3.5 model.               |
| Claude 3 Opus         | claude-3-opus-20240229     | Claude 3 optimized for platforms.      |
| Claude 3 Sonnet       | claude-3-sonnet-20240229   | Claude 3 for API integrations.         |
| Claude 3 Haiku        | claude-3-haiku-20240307    | Claude 3 for cloud platforms.          |


### Using the `LLM_Messages` Class for Different API Types

This part demonstrates how to use the `LLM_Messages` class to create and manage messages for different Large Language Model (LLM) APIs, namely OpenAI, Cohere, and Anthropic. 

We will:
1. Define the `LLM_Messages` class and the `message_models` dictionary.
2. Create instances of `LLM_Messages` for each API type.
3. Add system, human, and assistant messages.
4. Retrieve messages in different formats.
5. Export messages to JSON format.


In [ ]:
import json

# Define message models for different APIs
message_models = {
    'openai': {
        'roles': {
            'in': 'user',
            'out': 'assistant',
            'system': 'system'
        },
        'output_field': 'content'
    },
    'cohere': {
        'roles': {
            'in': 'USER',
            'out': 'CHATBOT',
            'system': 'SYSTEM'
        },
        'output_field': 'text'
    },
    'anthropic': {
        'roles': {
            'in': 'user',
            'out': 'assistant',
            'system': 'system'
        },
        'output_field': 'content'
    },
}

# Define the LLM_Messages class
class LLM_Messages:
    def __init__(self, api_type):
        if api_type in [ 'openai', 'cohere', 'anthropic' ]:
            self.api_type = api_type
            self.message_model=message_models[api_type]
        else:
            raise ValueError("Unsupported API type. Choose 'openai', 'cohere', or 'anthropic'.")

        self.messages = []

    def add_system_message(self, message):
        self.messages.append({"role": self.message_model['roles']['system'], self.message_model['output_field']: message})

    def add_human_message(self, message):
        self.messages.append({"role": self.message_model['roles']['in'], self.message_model['output_field']: message})

    def add_assistant_message(self, message):
        self.messages.append({"role": self.message_model['roles']['out'], self.message_model['output_field']: message})
        
    def add_prompt(self, message):
        self.add_human_message(message)

    def add_response(self, message):
        self.add_assistant_message(message)

    def get_messages(self):
        return self.messages
    
    def get_api_messages(self):
        if self.api_type == 'openai':
            return self.messages
        if self.api_type == 'cohere':
            prompt = ""
            for message in self.messages:
                prompt += f"{message['role']}: {message['text']}\n"
            return prompt + self.message_model['roles']['out'] + ": "
        if self.api_type == 'anthropic':
            prompt = ""
            for message in self.messages:
                prompt += f"{message['role']}: {message['text']}\n"
            return prompt + self.message_model['roles']['out'] + ": "
            
    def to_json(self):
        return json.dumps(self.get_messages(), indent=4)


In [ ]:
# Example usage
# Create instances for each API type
openai_messages = LLM_Messages(api_type='openai')
cohere_messages = LLM_Messages(api_type='cohere')
anthropic_messages = LLM_Messages(api_type='anthropic')

# Add messages for OpenAI
openai_messages.add_system_message("You are an assistant.")
openai_messages.add_human_message("What is the weather today?")
openai_messages.add_assistant_message("The weather today is sunny with a high of 25°C.")

# Add messages for Cohere
cohere_messages.add_system_message("You are a helpful chatbot.")
cohere_messages.add_human_message("Tell me a joke.")
cohere_messages.add_assistant_message("Why don't scientists trust atoms? Because they make up everything!")

# Add messages for Anthropic
anthropic_messages.add_system_message("You are an AI assistant.")
anthropic_messages.add_human_message("How do I bake a cake?")
anthropic_messages.add_assistant_message("To bake a cake, you need to follow a recipe with steps such as mixing ingredients, preheating the oven, and baking for a specified time.")

# Retrieve messages
print("OpenAI Messages:", openai_messages.get_messages())
print("Cohere Messages:", cohere_messages.get_messages())
print("Anthropic Messages:", anthropic_messages.get_messages())

# Retrieve API-specific message format
print("OpenAI API Messages:", openai_messages.get_api_messages())
print("Cohere API Messages:", cohere_messages.get_api_messages())
print("Anthropic API Messages:", anthropic_messages.get_api_messages())

# Export to JSON
print("OpenAI Messages JSON:", openai_messages.to_json())
print("Cohere Messages JSON:", cohere_messages.to_json())
print("Anthropic Messages JSON:", anthropic_messages.to_json())

# Worker Class

The `Worker` class is designed to process tasks based on the specified agent profiles. It includes methods for adding system messages, human messages, assistant messages, prompts, responses, and generating responses. It also manages task processing, clearing messages, clearing results, and resetting the worker.


In [ ]:
import gc
#from myllm import MyLLM
#from llm_messages import LLM_Messages

class Worker:
    def __init__(self, name, api_type, api_key, model, profile=None, has_memory=False):
        self.name = name
        self.api_type = api_type
        self.llm_instance = MyLLM(api_type=api_type, api_key=api_key, model=model)
        self.llm_messages = LLM_Messages(api_type=api_type)
        self.has_memory = has_memory
        self.system_message = ""
        self.results = []
        
        if profile is not None:
            self.profile = profile
            self.add_system_message(profile['system_message'])

    def add_system_message(self, message):
        self.system_message = message
        self.llm_messages.add_system_message(message)

    def add_human_message(self, message):
        self.llm_messages.add_human_message(message)

    def add_assistant_message(self, message):
        self.llm_messages.add_assistant_message(message)

    def add_prompt(self, message):
        self.llm_messages.add_prompt(message)
        
    def add_prompts(self, messages):
        for message in messages:
            self.llm_messages.add_prompt(message)

    def add_response(self, message):
        self.llm_messages.add_response(message)

    def generate_response(self, temperature=0.0, max_tokens=1000, is_message=False):
        messages = self.llm_messages.get_api_messages()
        response = self.llm_instance.generate_response(messages, temperature=temperature, max_tokens=max_tokens)
        if not is_message:
            self.add_response(response)
        return response

    def process_tasks(self, prompt=None, temperature=0.0, max_tokens=1000, is_message=False):
        if self.profile is None:
            raise ValueError("Profile is not set. Cannot process tasks.")
        work_result = []
        if prompt is not None:
            self.add_prompt(prompt)
        for task in self.profile['tasks']:
            self.add_prompt(task['task'])
            result = self.generate_response(temperature=temperature, max_tokens=max_tokens, is_message=is_message)
            work_result.append(result)
        if self.has_memory:
            self.results.extend(work_result)
        else:
            self.reset()
        return work_result
    
    def clear_messages(self):
        del self.llm_messages
        gc.collect()
        self.llm_messages = LLM_Messages(api_type=self.api_type)
        self.llm_messages.add_system_message(self.system_message)

    def clear_results(self):
        del self.results
        gc.collect()
        self.results = []

    def reset(self):
        self.clear_messages()
        self.clear_results()

# A proposal for a resolution of a problem described in a Github issue

The following diagram illustrates a structured workflow designed to manage and resolve a GitHub issue through a system of specialized agents, each assigned specific roles and tasks. This process begins with the identification of an issue and progresses through a series of steps involving various agents such as the Overviewer, Decider, Code Reasoner, DB Analyst, Code Generator, and Assistant. Each agent contributes its expertise, ensuring that the issue is comprehensively analyzed, tasks are efficiently assigned, solutions are accurately developed, and all relevant information is gathered for final resolution.

```mermaid
stateDiagram-v2
    [Issue] --> Overviewer
    Overviewer --> Decider
    Decider --> Code_Reasoner
    Decider --> DB_Analyst
    Decider --> Assistant
    Code_Reasoner --> Code_Generator
    Code_Generator --> [*]
    DB_Analyst --> [*]
    Assistant --> [*]
```

The diagram describes the workflow of handling a GitHub issue using a system of specialized agents, each with a specific role. Here's a step-by-step explanation of the diagram:

1. **Issue**:
    - The process starts with an issue being identified or received.

2. **Overviewer**:
    - The issue is first sent to the Overviewer.
    - The Overviewer is responsible for generating an overview of the issue by identifying the problem, setting objectives, providing specific details, and including relevant background information.

3. **Decider**:
    - After the Overviewer generates an overview, the information is passed to the Decider.
    - The Decider's role is to analyze the tasks required and decide which specialized agent should handle each task.

4. **Code Reasoner, DB Analyst, and Assistant**:
    - The Decider can choose to send tasks to the Code Reasoner, DB Analyst, or Assistant, depending on the nature of the tasks.
    - **Code Reasoner**: Responsible for documenting requirements, designing solutions, and analyzing the environment needed to run the solution.
    - **DB Analyst**: Responsible for database-related analyses and producing database instructions.
    - **Assistant**: Acts as a versatile helper, taking on tasks that do not specifically fall under the expertise of the Code Reasoner or DB Analyst.

5. **Code Generator**:
    - If the Decider assigns tasks to the Code Reasoner, the Code Reasoner may further pass specific sub-tasks to the Code Generator.
    - The Code Generator is responsible for writing the necessary code to implement solutions and developing tests for different cases, including edge cases.

6. **Final State**:
    - The workflow reaches its final state after the Code Generator, DB Analyst, or Assistant completes their respective tasks.
    - This end point signifies that all necessary information has been gathered and processed, leading to the final resolution or closure of the issue.


The workflow ensures that each aspect of the issue is handled by the appropriate specialized agent.

## Profiles Overview

### Overviewer Profile

The `Overviewer` profile is designed to generate an overview of a GitHub issue. It includes tasks for identifying and articulating the problem, setting objectives, setting sub-problems, providing specific details, and including relevant background information.


In [ ]:
Overviewer_profile = {
    "name": "Overviewer",
    "description": "Overviewer is an agent that generates an overview of a GitHub issue.",
    "system_message": "You are a system analyst and a problem solver.",
    "tasks": [
        {
            "task": """
            **Instructions for Articulating the Problem:**

                1. **Identify the Problem**: Clearly identify and articulate the problem you are addressing.

                2. **Clearly set the objective**: Clearly set the objective and the means to solve the problem.

                3. **Provide Specific Details and Context**: Offer specific details and context to ensure a comprehensive understanding of the problem.

                4. **Include Relevant Background Information**: Include any relevant background information that may help in understanding the problem.

                5. **Default Development Context**: If no specific development context is provided, assume the following:
                    - **Programming Language**: Python
                    - **Virtual Environment**: venv
                    - **Testing Frameworks**: pytest and vcrpy
                """
        },
        {
            "task": """
            **Instructions for Dividing the Problem into Sub-Problems:**
            1. **Divide the Problem**: Break down the main problem into smaller, manageable parts called sub-problems.
            2. **Identify Inputs and Outputs**: For each sub-problem, clearly identify the specific inputs required and the expected outputs.
            3. **Detailed Breakdown**: Provide a thorough and detailed breakdown of each sub-problem.
            4. **Sub-Problem Description**: Begin the description of each sub-problem with '#Sub-problem'.
            
            The output format must be JSON.
            
            ---

            **Example:**

            {
            "sub_problems": [
                {
                "id": 1,
                "title": "Sub-problem 1 Title",
                "inputs": "List the specific inputs needed for this sub-problem.",
                "outputs": "List the expected outputs from this sub-problem.",
                "description": "Provide a detailed explanation of the tasks, steps, or calculations involved."
                },
                {
                "id": 2,
                "title": "Sub-problem 2 Title",
                "inputs": "List the specific inputs needed for this sub-problem.",
                "outputs": "List the expected outputs from this sub-problem.",
                "description": "Provide a detailed explanation of the tasks, steps, or calculations involved."
                },
                {
                "id": N,
                "title": "Sub-problem N Title",
                "inputs": "List the specific inputs needed for this sub-problem.",
                "outputs": "List the expected outputs from this sub-problem.",
                "description": "Provide a detailed explanation of the tasks, steps, or calculations involved."
                }
            ]
            }

            ---

            Note that these sub-problems will be resolved by a machine, not a human.
            """
        }
    ]
}

### Decider Profile

The `Decider` profile is responsible for deciding which worker to consult based on the task at hand. It includes options for different agents and outputs the decision in JSON format.


In [ ]:
Decider_profile = {
    "name": "Decider",
    "description": "Decider is an agent that decides which worker to consult.",
    "system_message": "You are a problem solver and a wise decision maker.",
    "tasks": [
        {
            "task": """
                According to the task at hand, take a wise decision on which worker should perform it.
                Options:
                    1. Code Reasoner (Code Reasoner is an agent that documents the requirements, designs the solution and analyses the environment to run the solution)
                    2. Database Engineer (Database Engineer is an agent performs database analyses and produces database instructions)
                    3. Assistant (Choose Assistant as a versatile helper only when no other worker is suited for the task.)
                
                The output format must be JSON.
                
                ---
                
                **Example:**

                {
                "decision": "Code Reasoner"
                }
            """
        }
    ]
}

### Code Reasoner Profile

The `Code Reasoner` profile documents the requirements, designs the solution, and analyzes the environment needed to run the solution.

In [ ]:
Code_Reasoner_profile = {
    "name": "Code Reasoner",
    "description": "Code Reasoner is an agent that documents the requirements, designs the solution and analyses the environment to run the solution.",
    "system_message": "You are a system analyst.",
    "tasks": [
        {
            "task": """
                List all requirements and constraints for solving the problem.
                Include functional requirements, performance criteria, and any
                limitations or assumptions that need to be considered.
            """
        },
        {
            "task": """
                Create a high-level design for the solution. Outline the logic
                and steps needed to solve the problem.
            """
        },
        {
            "task": """
                Ensure all necessary software and libraries are installed. 
                Provide a list of required tools and instructions for setting up the development environment. 
                Verify that the environment is correctly configured for the task.
            """
        }
    ]
}

### Code Generator Profile

The `Code Generator` profile develops programming code and writes tests to cover different cases, including edge cases.


In [ ]:
Code_Generator_profile = {
    "name": "Code Generator",
    "description": "Code Generator is an agent that develops programming code.",
    "system_message": "You are a software developer.",
    "tasks": [
        {
            "task": """
                Your response must contain only the necessary code,
                with minimal and commented textual descriptions.
                Write the code to implement a solution for the sub-problem.

                The output format must be JSON.
                
                ---

                **Example:**

                {
                "output_files": [
                    {
                    "name": "some_output_file.py",
                    "content": "..."
                    },
                    ...
                    ]
                }
            """
        },
        {
            "task": """
                Write the tests to cover different cases, including edge cases.

                The output format must be JSON.
                
                ---

                **Example:**

                {
                "output_files": [
                    {
                    "name": "some_output_file.py",
                    "content": "..."
                    },
                    ...
                    ]
                }
            """
        }
    ]
}

### DB Engineer Profile

The `DB Engineer` profile performs database analyses and produces database instructions.


In [ ]:
DB_Engineer_profile = {
    "name": "DB_Engineer",
    "description": "DB_Engineer is an agent performs database analyses and produces database instructions.",
    "system_message": """
                    You are a database engineer. Your role typically includes tasks such as:
                    1. **Database Design**: Creating the structure of the database, including tables, relationships, and indexes.
                    2. **Database Construction**: Building the database according to the design, often using SQL (Structured Query Language) or other database management tools.
                    3. **Writing Instructions**: Developing queries, stored procedures, triggers, and other database components to ensure the database functions efficiently and meets the needs of the application or user.
    """,
    "tasks": [
        {
            "task": """
                Given the context, develop code for the task.
            """
        }
    ]
}

### Assistant Profile

The `Assistant` profile is a general-purpose helper that attempts to solve tasks to the best of its ability.


In [ ]:
Assistant_profile = {
    "name": "Assistant",
    "description": "Assistant is a general purpose helper.",
    "system_message": "You are a helpful assistant.",
    "tasks": [
        {
            "task": """
                According to the task at hand, make your best effort to help solve the task.
                
                The output format must be JSON.
                
                ---
                
                **Example:**

                {
                "resolution": "..."
                }
            """
        }
    ]
}

# Implementation example of the concepts presented above

In this section, we will delve into a practical implementation example to illustrate the concepts discussed previously.

In [ ]:
import os
import sys
import requests
import json
from dotenv import load_dotenv

def get_github_issue(owner, repo, issue_number, token):
    url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}"
    
    headers = {}
    if token:
        headers['Authorization'] = f'Bearer {token}'
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 403:
        raise Exception("API rate limit exceeded. Please try again later.")
    elif response.status_code == 404:
        raise Exception(f"Issue {issue_number} not found in {owner}/{repo}.")
    else:
        raise Exception(f"Failed to fetch issue {issue_number} from {owner}/{repo}. HTTP Status Code: {response.status_code}")

def get_data_from_url(url):
    splits = url.split('/')
    if len(splits) >= 5:
        return {
            'owner': splits[-4],
            'repo': splits[-3],
            'issue': splits[-1]
        }
    else:
        raise ValueError("Invalid URL format. Expected format: https://github.com/<owner>/<repo>/issues/<issue_number>")

def cleanJSON(s):
    s = s[next(idx for idx, c in enumerate(s) if c in "{["):]
    try:
        return json.loads(str(s))
    except json.JSONDecodeError as e:
        return json.loads(s[:e.pos])


def main(args):
    if not args:
        print("Usage: test.py <github_issue_url>")
        sys.exit(1)
    try:
        load_dotenv()

        openai_api_key=os.environ['OPENAI_API_KEY']
        cohere_api_key=os.environ['COHERE_API_KEY']
        anthropic_api_key=os.environ['ANTHROPIC_API_KEY']
        github_api_key=os.environ['GITHUB_ACCESS_TOKEN']
        
        url = args[0]
        issue_data = get_data_from_url(url)
        issue = get_github_issue(issue_data['owner'], issue_data['repo'], issue_data['issue'], github_api_key)
        issue_body = issue['body']

        api_type_in_use = 'cohere'
        model_in_use = 'command-r-plus'
        llm_api_key=cohere_api_key

        worker_overviewer = Worker("Overviewer", api_type=api_type_in_use, api_key=llm_api_key, model=model_in_use, profile=Overviewer_profile)
        worker_decider = Worker("Decider", api_type=api_type_in_use, api_key=llm_api_key, model=model_in_use, profile=Decider_profile)
        worker_db_engineer = Worker("DB_Engineer", api_type=api_type_in_use, api_key=llm_api_key, model=model_in_use, profile=DB_Engineer_profile)
        worker_code_reasoner = Worker("Code_Reasoner", api_type=api_type_in_use, api_key=llm_api_key, model=model_in_use, profile=Code_Reasoner_profile)
        worker_code_generator = Worker("Code_Generator", api_type=api_type_in_use, api_key=llm_api_key, model=model_in_use, profile=Code_Generator_profile)
        worker_assistant = Worker("Assistant", api_type=api_type_in_use, api_key=llm_api_key, model=model_in_use, profile=Assistant_profile)

        result = worker_overviewer.process_tasks(prompt=issue_body)
        
        working_base = []
        working_base.append(result[0])

        sub_problems = cleanJSON(result[1])
        
        for sp in sub_problems['sub_problems']:
            print("## " + str(sp['title']))
            message="**TASK**: " + str(sp['title']) + "\n"
            message+="inputs: " + str(sp['inputs']) + "\n"
            message+="outputs: " + str(sp['outputs']) + "\n"
            message+="description: " + str(sp['description']) + "\n"
            
            result_decision = worker_decider.process_tasks(prompt=message)
            decision_json = cleanJSON(result_decision[0])

            print("# " + str(decision_json['decision']))
            
            if decision_json['decision'] == "Code Reasoner":
                worker_code_reasoner.add_prompts(working_base)
                rcr = worker_code_reasoner.process_tasks(prompt=message)
                worker_code_generator.add_prompts(working_base)
                worker_code_generator.add_prompt(message)
                worker_code_generator.add_prompts(rcr)
                result = worker_code_generator.process_tasks()
            elif decision_json['decision'] == "Database Engineer":
                worker_db_engineer.add_prompts(working_base)
                worker_db_engineer.add_prompt(message)
                result = worker_db_engineer.process_tasks()
            else:
                worker_assistant.add_prompts(working_base)
                worker_assistant.add_prompt(message)
                result = worker_assistant.process_tasks()
            
            print(result)

    except Exception as e:
        print(f"Error: {e}")
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        sys.exit(1)

if __name__ == "__main__":
    main(sys.argv[1:])